<a href="https://colab.research.google.com/github/annikarichardson2024/ChatBot/blob/main/ChatbotEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [34]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model
import numpy as np


In [35]:
# data
questions = ['Hello', 'How are you?', 'What is your name?','Goodbye']
answers = ['Hey there!','Very well.', 'My name is bot.','Goodbye!']


In [36]:
# create tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(questions + answers)

In [37]:
# create sequences
sequences_q = tokenizer.texts_to_sequences(questions)
sequences_a = tokenizer.texts_to_sequences(answers)

In [22]:
# pad the sequences
max_length = max(max(len(seq) for seq in sequences_q), max(len(seq) for seq in sequences_a))
padded_q = tf.keras.preprocessing.sequence.pad_sequences(sequences_q, maxlen=max_length, padding='post')
padded_a = tf.keras.preprocessing.sequence.pad_sequences(sequences_a, maxlen=max_length, padding='post')



In [39]:
# set parameters
embedding_dim = 2000
units = 1000
vocab_size = 5000


In [40]:
# encoder model
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs, state_h, state_c = LSTM(units, return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# decoder model
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [41]:
# seq2seq model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# compiling the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


In [42]:
decoder_input_data = np.zeros_like(padded_a)
decoder_input_data[:, 0] = 1

# Train  model
num_epochs = 100
model.fit([padded_q, decoder_input_data], np.expand_dims(padded_a, -1), batch_size=2, epochs=num_epochs)


Epoch 1/100
2/2 [==============================] - 3s 74ms/step - loss: 8.4471 - accuracy: 0.1875
Epoch 2/100
2/2 [==============================] - 0s 24ms/step - loss: 7.9220 - accuracy: 0.4375
Epoch 3/100
2/2 [==============================] - 0s 82ms/step - loss: 5.9157 - accuracy: 0.4375
Epoch 4/100
2/2 [==============================] - 0s 23ms/step - loss: 3.9507 - accuracy: 0.4375
Epoch 5/100
2/2 [==============================] - 0s 23ms/step - loss: 3.3060 - accuracy: 0.4375
Epoch 6/100
2/2 [==============================] - 0s 23ms/step - loss: 2.7970 - accuracy: 0.3125
Epoch 7/100
2/2 [==============================] - 0s 23ms/step - loss: 2.9545 - accuracy: 0.1250
Epoch 8/100
2/2 [==============================] - 0s 22ms/step - loss: 2.1314 - accuracy: 0.4375
Epoch 9/100
2/2 [==============================] - 0s 22ms/step - loss: 1.9249 - accuracy: 0.4375
Epoch 10/100
2/2 [==============================] - 0s 22ms/step - loss: 1.8393 - accuracy: 0.4375
Epoch 11/100
2/2 [=

In [43]:
# function to predict answers
def preprocess(input_text):
    sequence = tokenizer.texts_to_sequences([input_text])
    padded_sequence = tf.keras.preprocessing.sequence.pad_sequences(sequence, maxlen=max_length, padding='post')
    return padded_sequence

def generate(input_sequence):
    response_sequence = np.zeros((1, max_length))
    response_sequence[0, 0] = 1
    for i in range(1, max_length):
        prediction = model.predict([input_sequence, response_sequence]).argmax(axis=2)
        response_sequence[0, i] = prediction[0, i-1]
        if prediction[0, i-1] == 2:
            break
    return response_sequence

def sequence(sequence):
    return ' '.join(tokenizer.index_word.get(i, '') for i in sequence if i > 2)

In [48]:
# putting it all together to get responses

def chat(input_text):
    input_sequence = preprocess(input_text)
    response_sequence = generate(input_sequence)
    response_text = sequence(response_sequence[0])
    return response_text

In [51]:
# Interactive chat
print("Say something! Type 'i quit' or 'goodbye' or 'i am done' to exit.")
while True:
    question = input('You: ')
    if question.lower() == 'i quit':
        break
    if question.lower() == 'goodbye':
        break
    if question.lower() == 'i am done':
        break
    response = chat(question)
    print(f"Chatbot: {response}")



Say something! Type 'i quit' or 'goodbye' or 'i am done' to exit.
You: I quit
